# Landmark Detection using Transfer Learning with MobileNetV2

This notebook implements a deep learning model to classify famous landmarks. We will use transfer learning on a pre-trained MobileNetV2 model to achieve high accuracy with a relatively small dataset.

## 1. Setup and Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os

## 2. Define Parameters and Load Data

In [ ]:
# Define paths and parameters
DATA_DIR = '../data/'
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 10

# Check if data directory exists
if not os.path.exists(DATA_DIR) or not os.listdir(DATA_DIR):
    print(f"Error: Data directory '{DATA_DIR}' is empty or does not exist.")
    print("Please download the landmark dataset and organize it into class-specific subdirectories inside 'data/'.")
else:
    print("Data directory found.")

In [ ]:
# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Use 20% of data for validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create data generators
train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

## 3. Build the Model using Transfer Learning

In [ ]:
# Load the MobileNetV2 base model, pre-trained on ImageNet, without the top classification layer
base_model = MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
)

# Freeze the layers of the base model
base_model.trainable = False

# Get the number of classes from the generator
num_classes = len(train_generator.class_indices)

# Add a custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

## 4. Train the Model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

## 5. Evaluate the Model

In [ ]:
# Plot training & validation accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 6. Save the Model

In [ ]:
# Save the entire model to a HDF5 file.
model.save('../saved_model/landmark_detector.h5')
print("Model saved to ../saved_model/landmark_detector.h5")

## 7. Next Steps: Fine-Tuning

For even better performance, you can unfreeze some of the top layers of the base model and train again with a very low learning rate. This process, known as fine-tuning, allows the model to adapt its learned features more closely to the landmark dataset.